<a href="https://colab.research.google.com/github/renataeugenia/ChallengeAlura_DataScience/blob/main/Challenge_Alura_DataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Alura Data Science - 1º semana 21/11/2022

In [34]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [35]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [36]:
import findspark
findspark.init()

In [37]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Challenge Alura") \
    .getOrCreate()

In [38]:
spark

**card: descompactando** **arquivos**

In [39]:
import zipfile

In [40]:
zipfile.ZipFile('/content/drive/MyDrive/Challenge_Alura/semana-1.zip', 'r').extractall('/content/drive/MyDrive/Challenge_Alura')

In [41]:
path = '/content/drive/MyDrive/Challenge_Alura/dataset_bruto.json'
dataset_bruto = spark.read.json(path)

**card: explorando base de dados**

In [42]:
dataset_bruto.count()

89083

In [43]:
dataset_bruto.limit(5).toPandas()

,anuncio,imagens,usuario
0,"(0, [], [16], [0], [], (Centro, 20061003, Rio ...","[(39d6282a-71f3-47bc-94aa-909351ecd881, https:...","(9d44563d-3405-4e84-9381-35b7cf40a9a4, Frank)"
1,"(0, [], [14], [0], [], (Centro, 20051040, Rio ...","[(23d2b3ab-45b0-47f2-a27f-ffbe066c2c38, https:...","(36245be7-70fe-40cd-84d4-70a57dc8f976, Caroline)"
2,"(0, [1026], [1026], [0], [], (Maria da Graça, ...","[(1da65baa-368b-490f-bef7-1834b9a428c1, https:...","(9dc415d8-1397-4d8d-8425-b8310fad309d, Oliver)"
3,"(0, [120], [120], [0], [Portão eletrônico, Con...","[(79b542c6-49b4-4f06-80b5-ef4fcdf98f76, https:...","(9911a2df-f299-4a73-a384-281417abaf69, Matthew)"
4,"(0, [3], [3], [0], [], (São Cristóvão, 2093167...","[(e2bc497b-6510-4731-8942-97b3d3cdab5e, https:...","(240a7aab-12e5-4069-9a2c-9dca4c4f9c68, Jude)"


In [44]:
dataset_bruto.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

**card: Transformar cada campo da coluna "anuncio" em uma coluna** **separada** 

In [45]:
anuncio = dataset_bruto\
.select('anuncio.*')

In [46]:
anuncio.limit(5).toPandas()

,andar,area_total,area_util,banheiros,caracteristicas,endereco,id,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,valores
0,0,[],[16],[0],[],"(Centro, 20061003, Rio de Janeiro, Rio de Jane...",47d553e0-79f2-4a46-9390-5a3c962740c2,[0],[0],Usado,Outros,Comercial,[1],"[(260, 107, Venda, 10000)]"
1,0,[],[14],[0],[],"(Centro, 20051040, Rio de Janeiro, Rio de Jane...",b6ffbae1-17f6-4870-9950-e998ac1c8d6a,[0],[],Usado,Outros,Comercial,[0],"[(260, 107, Venda, 10000)]"
2,0,[1026],[1026],[0],[],"(Maria da Graça, 20785360, Rio de Janeiro, Rio...",1fb030a5-9e3e-4a1d-93cd-cd2d8a215e11,[0],[],Usado,Outros,Comercial,[0],"[(None, 1613, Venda, 10000)]"
3,0,[120],[120],[0],"[Portão eletrônico, Condomínio fechado]","(Campo Grande, 23095071, Rio de Janeiro, Rio d...",ac6cb348-69d6-45af-9589-dc34099370d8,[0],[],Usado,Outros,Residencial,[],"[(80, None, Venda, 10000)]"
4,0,[3],[3],[0],[],"(São Cristóvão, 20931670, Rio de Janeiro, Rio ...",e032b908-ef42-4d4a-8125-eba4792bacbe,[0],[],Usado,Outros,Residencial,[],"[(0, 0, Venda, 5000)]"


**card: Filtrar a base de dados**

In [47]:
anuncio\
.select('tipo_uso')\
.groupby('tipo_uso')\
.count()\
.show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



In [48]:
anuncio = anuncio\
    .where("tipo_uso == 'Residencial'")
    

In [49]:
anuncio.show(10)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|     [120]|    [120]|      [0]|[Portão eletrônic...|{Campo Grande, 23...|ac6cb348-69d6-45a...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{80, null, Venda...|
|    0|       [3]|      [3]|      [0]|                  []|{São Cristóvão, 2...|e032b908-ef42-4d4...|    [0]|    []|       Usado|      Outros|Residencial|  []|[{0, 0, Venda, 50...|
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-4

In [50]:
anuncio\
    .groupby('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros| 7517|
| Apartamento|66797|
|        Casa|10227|
+------------+-----+



In [51]:
anuncio = anuncio\
    .filter("tipo_unidade == 'Apartamento'")
    

In [52]:
anuncio.show(10)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

In [53]:
anuncio\
    .groupby('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
|  Lançamento|  235|
+------------+-----+



In [54]:
anuncio = anuncio\
    .where("tipo_anuncio == 'Usado'")
   

In [55]:
anuncio.show(10)

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|      [43]|     [43]|      [1]|[Academia, Churra...|{Paciência, 23585...|d2e3a3aa-09b5-45a...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{245, null, Vend...|
|    2|      [42]|     [42]|      [1]|[Churrasqueira, P...|{Paciência, 23585...|085bab2c-87ad-452...|    [2]|    []|       Usado| Apartamento|Residencial| [1]|[{0, 0, Venda, 15...|
|    1|      [41]|     [41]|      [1]|[Portaria 24h, Co...|{Guaratiba, 23036...|18d22cbe-1b86-4

**card: Transformar as colunas dos cômodos dos imóveis de listas para inteiros**
(transforme os dados das colunas "quartos", "suites", "banheiros", "vaga", "area_total" e "area_util" de listas para inteiros.)
- analisando os elementos dentro de cada coluna.
- aplicando o list comprehension para tranformar os dados em inteiros.


In [56]:
from pyspark.sql import functions as f

In [57]:
caracteristicas = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [58]:
for col in caracteristicas:
  anuncio\
  .select(f.size(f.col(col)).alias(col))\
  .groupBy(col)\
  .count()\
  .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|66562|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|61008|
|     0| 5554|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|66562|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|63545|
|   0| 3017|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|57368|
|         0| 9194|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|66562|
+---------+-----+



In [59]:
anuncio = anuncio\
    .select([f.col(c)[0].alias(c) if c in caracteristicas else c for c in anuncio.columns])
    

In [60]:
anuncio.show(truncate=False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |endereco                                                                                                                             |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+--------------------------------------------------------------------------

In [61]:
anuncio.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- eleme

**card: Tratamento de informações sobre localização**
- analisar a coluna endereco e transformar apenas as informações sobre bairro e zona em colunas de nosso DataFrame.

In [62]:
anuncio\
    .select('andar', 'area_total', 'area_util', 'banheiros', 'caracteristicas', 'endereco.*', 'id', 'quartos', 'suites', 'tipo_anuncio', 'tipo_unidade', 'tipo_uso', 'vaga', 'valores'   )\
    .show(truncate=False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+--------+--------------+--------------+----------+----------+----+-------------------------+----------+------------------------------------+-------+------+------------+------------+-----------+----+----------------------------+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |bairro                  |cep     |cidade        |estado        |latitude  |longitude |pais|rua                      |zona      |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|valores                     |
+-----+----------+---------+---------+--------------------------------------------------------------------------------------

In [63]:
anuncio_novo = anuncio\
    .select('andar', 'area_total', 'area_util', 'banheiros', 'caracteristicas', 'endereco.*', 'id', 'quartos', 'suites', 'tipo_anuncio', 'tipo_unidade', 'tipo_uso', 'vaga', 'valores'   )\
    .drop('cep', 'cidade', 'estado','latitude','longitude','pais','rua')
   

In [64]:
anuncio_novo.show()

+-----+----------+---------+---------+--------------------+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|              bairro|      zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    3|        43|       43|        1|[Academia, Churra...|           Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|
|    2|        42|       42|        1|[Churrasqueira, P...|           Paciência|Zona Oeste|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|
|    1|        41|       41|        1|[P

In [65]:
anuncio_novo.limit(10).toPandas()

,andar,area_total,area_util,banheiros,caracteristicas,bairro,zona,id,quartos,suites,tipo_anuncio,tipo_unidade,tipo_uso,vaga,valores
0,3,43,43,1,"[Academia, Churrasqueira, Playground, Salão de...",Paciência,Zona Oeste,d2e3a3aa-09b5-45a0-9dcd-918847cd3ca3,2,NaN,Usado,Apartamento,Residencial,1,"[(245, None, Venda, 15000)]"
1,2,42,42,1,"[Churrasqueira, Playground, Salão de festas, C...",Paciência,Zona Oeste,085bab2c-87ad-452a-bd0f-8b1451509f84,2,NaN,Usado,Apartamento,Residencial,1,"[(0, 0, Venda, 15000)]"
2,1,41,41,1,"[Portaria 24h, Condomínio fechado, Playground,...",Guaratiba,Zona Oeste,18d22cbe-1b86-4764-8def-70c615f15a61,2,NaN,Usado,Apartamento,Residencial,1,"[(0, 0, Venda, 20000)]"
3,3,43,43,1,"[Churrasqueira, Piscina, Playground, Salão de ...",Cosmos,Zona Oeste,bed8a354-9317-4426-b27e-1c798f864271,2,NaN,Usado,Apartamento,Residencial,0,"[(285, None, Venda, 20000)]"
4,2,43,43,1,"[Academia, Churrasqueira, Playground, Salão de...",Guaratiba,Zona Oeste,12a13315-d67f-48f0-9497-017b83252781,2,NaN,Usado,Apartamento,Residencial,1,"[(245, None, Venda, 15000)]"
5,3,43,43,1,"[Churrasqueira, Piscina, Playground, Salão de ...",Cosmos,Zona Oeste,a2e6d7a5-0ff0-484d-b3d8-3a8f15e2d80e,2,0.0,Usado,Apartamento,Residencial,0,"[(285, None, Venda, 20000)]"
6,3,43,43,1,"[Churrasqueira, Playground, Salão de festas, C...",Paciência,Zona Oeste,a6e3173b-c950-4db4-9b7b-80c44bd75e90,2,NaN,Usado,Apartamento,Residencial,1,"[(250, None, Venda, 15000)]"
7,3,43,43,1,"[Churrasqueira, Playground, Salão de festas, C...",Guaratiba,Zona Oeste,2e6e5dfb-206c-4968-944b-ea4c3918b50d,2,0.0,Usado,Apartamento,Residencial,1,"[(245, None, Venda, 15000)]"
8,3,43,43,1,"[Churrasqueira, Playground, Salão de festas, C...",Paciência,Zona Oeste,99f8d0f9-95a4-4613-a55d-c949e7a73e90,2,0.0,Usado,Apartamento,Residencial,1,"[(245, None, Venda, 15000)]"
9,4,43,43,1,"[Churrasqueira, Playground, Salão de festas, C...",Guaratiba,Zona Oeste,b3f44c1a-2d50-4d56-b1fb-a94d59b55ab8,2,NaN,Usado,Apartamento,Residencial,1,"[(240, None, Venda, 17999)]"


**card: Transformar cada campo da coluna "valores" em uma coluna separada**

In [66]:
anuncio_novo.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)



In [67]:
anuncio_novo = anuncio_novo\
    .withColumn('condominio', anuncio_novo.valores.condominio[0])\
    .withColumn('iptu', anuncio_novo.valores.iptu[0])\
    .withColumn('tipo', anuncio_novo.valores.tipo[0])\
    .withColumn('valor', anuncio_novo.valores.valor[0])\
    .drop(anuncio_novo.valores)


In [68]:
anuncio_novo.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [69]:
anuncio_novo.show()

+-----+----------+---------+---------+--------------------+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|              bairro|      zona|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+----------+--------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+
|    3|        43|       43|        1|[Academia, Churra...|           Paciência|Zona Oeste|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|       245|null|Venda|15000|
|    2|        42|       42|        1|[Churrasqueira, P...|           Paciência|Zona Oeste|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|         0|   0|Venda|15000|
|    

**card: Tratamento para a coluna de valores**

In [71]:
anuncio_novo = anuncio_novo\
    .select('*')\
    .where("tipo == 'Venda'")

In [72]:
anuncio_novo.count()

66348

In [74]:
anuncio_novo.show(truncate=False)

+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+----------+------------------------------------+-------+------+------------+------------+-----------+----+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|caracteristicas                                                                                                                                   |bairro                  |zona      |id                                  |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|condominio|iptu|tipo |valor|
+-----+----------+---------+---------+--------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+----------+------------------------------------+-------+------+------------+------------+-----------

**card: Salvar os dados no formato parquet**

In [77]:
anuncio_novo.write.parquet(
    path='/content/drive/MyDrive/Challenge_Alura/parquet',
    mode='overwrite'
    )

**card: Salvar os dados no formato csv**
- *Antes transformar a coluna **caracteristicas**, de Array para Strings para salvar o aquivo em csv.*

In [78]:
from pyspark.sql.types import StringType

In [79]:
anuncio_novo = anuncio_novo\
    .withColumn('caracteristicas', anuncio_novo.caracteristicas.cast(StringType()))

In [80]:
anuncio_novo.write.csv(
    path='/content/drive/MyDrive/Challenge_Alura/csv',
    mode='overwrite',
    sep=';',
    header=True
)

**card: Comparar o desempenho da leitura**

In [81]:
%%time
anuncio_novo_parquet = spark.read.parquet(
    '/content/drive/MyDrive/Challenge_Alura/parquet'
)

CPU times: user 3.67 ms, sys: 3.17 ms, total: 6.84 ms
Wall time: 249 ms


In [85]:
%%time
anuncio_novo_csv = spark.read.csv(
    path='/content/drive/MyDrive/Challenge_Alura/csv',
    sep=';',
    inferSchema=True,
    header=True)

CPU times: user 10.2 ms, sys: 2.13 ms, total: 12.3 ms
Wall time: 1.32 s


**Finalizando a sessão Spark**

In [ ]:
spark.stop()